In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.simplefilter("ignore", UserWarning)
import pandas as pd
import csv
import sys
import array
import xarray as xr
import cmocean.cm as cmocean
def write_with_byte_inversion(f, tab):
    tab2write = array.array('f', tab.T.flatten())
    if sys.byteorder == 'little':
        tab2write.byteswap()
    f.write(tab2write)

dtype=np.float32
#topo_adjust to topo wi
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from matplotlib.ticker import FuncFormatter
import pathlib
from pathlib import Path
import os
import netCDF4
from netCDF4 import Dataset as nc
import glob
from glob import glob
import glidertools as gt
import gsw

In [ ]:
ds_work = xr.open_dataset('ds_workfinal.nc')
ds_work

# grid depth

In [ ]:
# grid
depth = np.arange(0,1000,1)
latitude = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.latitude_gsm, bins=depth)
longitude = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.longitude_gsm, bins=depth)
ctd_pressure = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.ctd_pressure, bins=depth)
o2 = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.aanderaa4831_dissolved_oxygen, bins=depth)
salinity = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.absolute_salinity, bins=depth)
temp = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.conservative_temperature, bins=depth)
sol = gt.grid_data(ds_work.dives, ds_work.ctd_depth, ds_work.dissolved_oxygen_sat, bins=depth)

# grid on density for 1 profile 

In [ ]:
temp_profile = temp.isel(dives=100)
salt_profile = salinity.isel(dives=100)

density_profile = gsw.sigma0(salt_profile, temp_profile)

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3))

ax[0].plot(temp_profile, temp.ctd_depth)
ax[1].plot(salt_profile, salinity.ctd_depth)
ax[2].plot(density_profile, salinity.ctd_depth)

for a in ax:
    a.set_ylim(1000, 0)

In [ ]:
den_grid = np.arange(25, 28.01, 0.01)

In [ ]:
from scipy.interpolate import griddata

In [ ]:
ind_sort = np.argsort(density_profile.data)
den_sort = np.sort(density_profile.data)

temp_sort = griddata(den_sort, temp_profile[ind_sort], den_grid)
salt_sort = griddata(den_sort, salt_profile[ind_sort], den_grid)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(6,3))

ax[0].plot(temp_sort, den_grid)
ax[1].plot(salt_sort, den_grid)

for a in ax:
    a.set_ylim(28, 25)

# grid density for all variables

In [ ]:
den_grid = np.arange(24, 28.01, 0.01)

In [ ]:
density = gsw.sigma0(salinity, temp)

In [ ]:
density.plot()

In [ ]:
def grid_to_den(density, var, den_grid):
    
    var_grid = np.ndarray([var.dives.size, den_grid.size])
    
    for v in range(var.dives.size):
        
        density_profile = density.isel(dives=v)
        var_profile = var.isel(dives=v)
    
        ind_sort = np.argsort(density_profile.data)
        den_sort = np.sort(density_profile.data)

        var_grid[v] = griddata(den_sort, var_profile[ind_sort], den_grid)
        
    return var_grid

In [ ]:
ox_sol = gsw.O2sol(salinity, temp, ctd_pressure, longitude, latitude)

In [ ]:
temp_den_grid = grid_to_den(density, temp, den_grid)
sal_den_grid = grid_to_den(density, salinity, den_grid)
ox_den_grid = grid_to_den(density, o2, den_grid)
sol_den_grid = grid_to_den(density, ox_sol, den_grid)

# results variables against density

In [ ]:
fig,ax = plt.subplots(figsize=(10,4))

img=ax.pcolormesh(density.dives, den_grid, temp_den_grid.T)

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('density', fontsize=12)
plt.title('Temperature')

ax.invert_yaxis()

plt.colorbar(img)

In [ ]:
fig,ax = plt.subplots(figsize=(10,4))

img=ax.pcolormesh(density.dives, den_grid, sal_den_grid.T)

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('density', fontsize=12)
plt.title('Salinity')

ax.invert_yaxis()

plt.colorbar(img)

In [ ]:
fig,ax = plt.subplots(figsize=(10,4))

img=ax.pcolormesh(density.dives, den_grid, ox_den_grid.T)

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('density', fontsize=12)
plt.title('Oxygen')

ax.invert_yaxis()

plt.colorbar(img)

In [ ]:
fig,ax = plt.subplots(figsize=(10,4))

img=ax.pcolormesh(density.dives, den_grid, sol_den_grid.T)

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('density', fontsize=12)
plt.title('Solubility using gsw')

ax.invert_yaxis()

plt.colorbar(img)

In [ ]:
AOU_den_grid = sol_den_grid - ox_den_grid

In [ ]:
fig,ax = plt.subplots(figsize=(10,4))

img=ax.pcolormesh(density.dives, den_grid, AOU_den_grid.T)

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('density', fontsize=12)
plt.title('AOU')


ax.invert_yaxis()

plt.colorbar(img)

# solubility

In [ ]:
ox_sol = gsw.O2sol(salinity, temp, ctd_pressure, longitude, latitude)

In [ ]:
ox_sol

In [ ]:
x = np.array(ds_work.dives)  # ensures these are arrays
y = np.array(ds_work.ctd_depth)

gt.plot(x, y, ds_work.dissolved_oxygen_sat)


In [ ]:
gt.plot(x,y, ds_work.aanderaa4831_dissolved_oxygen)

In [ ]:
# pcolormesh graph
plt.pcolormesh(ox_sol.dives, ox_sol.ctd_depth, ox_sol)

# Add colorbar
plt.colorbar(label='Solubility Oxygen (micromoles/kg)')

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('Depth (m)', fontsize=12)

# Invert y-axis
plt.gca().invert_yaxis()

In [ ]:
# pcolormesh graph
plt.pcolormesh(sol.dives, sol.ctd_depth, sol)

# Add colorbar
plt.colorbar(label='Solubility Oxygen (micromoles/kg)')

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('Depth (m)', fontsize=12)

# Invert y-axis
plt.gca().invert_yaxis()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming ox_sol and sol are your data arrays

# Calculate the difference
difference =  sol - ox_sol

# Plot the difference using pcolormesh
plt.pcolormesh(ox_sol.dives, ox_sol.ctd_depth, difference)

# Add colorbar
plt.colorbar(label='Diff Solubility Oxygen (micromoles/kg)')

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('Depth (m)', fontsize=12)
plt.title('solubility dataset - TEOS-10 gsw')

# Invert y-axis
plt.gca().invert_yaxis()

# Show plot
plt.show()


In [ ]:
AOU = sol - o2
AOU

In [ ]:
# pcolormesh graph
plt.pcolormesh(AOU.dives, AOU.ctd_depth, AOU)

# Add colorbar
plt.colorbar(label='AOU (micromoles/kg)')

# Add labels and title
plt.xlabel('Dives', fontsize=12)
plt.ylabel('Depth (m)', fontsize=12)

# Invert y-axis
plt.gca().invert_yaxis()